In [36]:
import pandas as pd
import numpy as np
import ast
from matplotlib import pyplot as plt
import math
import time

In [37]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import OneHotEncoder

In [38]:
# So we can see all of the columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 600)

In [259]:
# Read in data
df_train = pd.read_csv("datas/train.csv", parse_dates=['host_since', 'first_review', 'last_review'])
df_test = pd.read_csv("datas/test.csv", parse_dates=['host_since', 'first_review', 'last_review'])

In [260]:
df = pd.concat([df_train, df_test])
# df = df_train

In [278]:
def join_amenities_column(amenities_column):
    """
    Joins strings within each row of the amenities column with ', ' and 
    separates rows with '; '.
    """
    def clean_strings(strings):
        # Replace invalid Unicode characters in each string
        return [s.encode('utf-8', 'replace').decode('utf-8') for s in strings]
    
    row_strings = amenities_column.apply(
        lambda x: ", ".join(clean_strings(x)) if isinstance(x, list) else ""
    )
    all_amenities = "; ".join(row_strings)
    return all_amenities

result = join_amenities_column(df['amenities'])
output_file_path = "amenities.txt"
with open(output_file_path, "w") as file:
    file.write(result)

In [261]:
df.head()

,name,description,property_type,price,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,has_availability,availability_30,availability_60,availability_90,availability_365,instant_bookable,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,reviews,id
0,Bed-Stuy 2 Bed/2 Bath - Renovated,"Welcome to Bed-Stuy, Brooklyn! Our newly renov...",Entire rental unit,4.0,Bedford-Stuyvesant,Brooklyn,40.684560,-73.939870,2015-05-23,within a day,100.0,100.0,True,2.0,2.0,"['email', 'phone']",True,True,1,1,0,0,Entire home/apt,4,2.0,2 baths,2.0,2.0,"[""Dishwasher"", ""Single level home"", ""Stove"", ""...",True,12,42,70,70,False,30,1125,34,5,1,2019-04-28,2024-08-10,5.00,5.00,4.97,5.0,5.0,4.71,4.94,0.52,"Barry's place was perfect. It was cute, modern...",NaN
1,Victorian Flatbush Oasis,"Lovely nonsmoking annex in Brooklyn's ""secret ...",Private room in rental unit,3.0,Flatbush,Brooklyn,40.638991,-73.965739,2023-09-14,within an hour,100.0,98.0,True,1.0,1.0,"['email', 'phone']",True,True,1,0,1,0,Private room,2,1.0,1 private bath,1.0,2.0,"[""Dishwasher"", ""Dining table"", ""Mini fridge"", ...",True,10,20,49,324,False,1,29,30,30,6,2024-01-13,2024-09-02,4.83,4.87,4.93,4.8,4.9,4.90,4.63,3.81,I booked this place last minute to attend a fu...,NaN
2,Bay Ridge Metroscape: Inviting NYC Studio Living,This studio presents unparalleled convenience ...,Entire rental unit,3.0,Fort Hamilton,Brooklyn,40.618810,-74.032380,2022-07-31,within an hour,100.0,100.0,False,52.0,55.0,"['email', 'phone']",True,True,52,50,2,0,Entire home/apt,2,1.0,1 bath,0.0,1.0,"[""Cleaning products"", ""Elevator"", ""Mini fridge...",True,17,44,70,146,True,1,29,5,5,2,2024-06-27,2024-08-17,4.60,4.80,4.20,4.8,4.8,4.80,4.20,2.14,Great spot! Little far out but overall a great...,NaN
3,"New HDTV room, 20 minutes to Manhattan #724",- Furnished room in a newly renovated apartmen...,Private room in rental unit,0.0,Crown Heights,Brooklyn,40.673970,-73.953990,2012-08-11,within an hour,99.0,23.0,False,727.0,1336.0,"['email', 'phone']",True,True,719,0,719,0,Private room,1,1.5,1.5 baths,4.0,1.0,"[""Kitchen"", ""Hot water"", ""Dedicated workspace""...",True,0,0,0,111,False,30,365,0,0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Just What You Were Looking For! Pets Allowed,This modern property in Manhattan is just step...,Room in hotel,2.0,Midtown,Manhattan,40.747180,-73.985390,2014-12-23,within an hour,93.0,95.0,False,707.0,2453.0,"['email', 'phone']",True,True,73,0,0,0,Hotel room,2,1.0,1 bath,1.0,1.0,"[""TV"", ""Smoke alarm"", ""Paid parking off premis...",True,4,13,22,241,True,1,1125,0,0,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
df.loc[df['beds'] == 0, 'beds'] = df.loc[df['beds'] == 0, 'bedrooms']

In [263]:
df['amenities'] = df['amenities'].apply(ast.literal_eval)
df['host_verifications'] = df['host_verifications'].apply(ast.literal_eval)

In [264]:
df['amenities_count'] = df['amenities'].apply(len)
df['description_length'] = df['description'].str.len()
df['name_length'] = df['name'].str.len()

In [265]:
# Are bathrooms shared?
df["shared_bathrooms"] = df['bathrooms_text'].str.contains('Shared|shared', case=False)

In [266]:
# Fancier amenities to look for
fancier_items_exact = ["Washer", "Dryer", "Dishwasher", "Freezer", "BBQ grill", "Free parking on premises",
                      "Paid parking on premises", "Free street parking", "City skyline view", "Shared beach access",
                      "Courtyard view", "Oven", "Radiant heating", "Dedicated workspace", "Fire extinguisher",
                      "Carbon monoxide alarm", "Outdoor furniture"]
fancier_items_contains = ["Hot Tub", "Pool", "Gym", "Balcony", "Backyard", "Beachfront", "Resort", "Sound system"]

# Check for exact matches (case insensitive)
def has_exact_item(row, item):
    lower_amenities = [amenity.lower() for amenity in row]
    return item.lower() in lower_amenities

# Check for substring matches (case insensitive)
def has_contained_item(row, item):
    lower_amenities = [amenity.lower() for amenity in row]
    return any(item.lower() in amenity for amenity in lower_amenities)

# Create columns for exact matches
for item in fancier_items_exact:
    df[f'has_{item.lower().replace(" ", "_")}'] = df['amenities'].apply(lambda x: has_exact_item(x, item))

# Create columns for substring matches
for item in fancier_items_contains:
    df[f'has_{item.lower().replace(" ", "_")}'] = df['amenities'].apply(lambda x: has_contained_item(x, item))

In [267]:
# Drop some unwanted columns
str_cols = [
    "name", "description", "reviews", "bathrooms_text", "amenities", "host_verifications", 
    # "property_type"
]
df_simple = df.drop(columns=str_cols)
df_simple = df_simple.drop(columns=[
    "has_availability",
    # "neighbourhood_cleansed",
    # "neighbourhood_group_cleansed"
])

In [268]:
# Datetime columns to ints
datetime_cols = ["host_since", "first_review", "last_review"]
for col in datetime_cols:
    # Store the median of non-NaT values
    median_timestamp = df_simple[col].dropna().astype("int64").median()
    # Convert to integers, NaT will become negative values
    df_simple[col] = df_simple[col].astype("int64")
    # Replace negative values (former NaT) with the median
    df_simple.loc[df_simple[col] < 0, col] = median_timestamp

In [269]:
# Fill missing values for boolean features
bool_cols = ["shared_bathrooms", "host_is_superhost"]
df_simple.fillna({col: df_simple[col].mode()[0] for col in bool_cols}, inplace=True)
df_simple[bool_cols] = df_simple[bool_cols].infer_objects()

/var/folders/wr/xdjfb2c1569g2d6hjp8hhhbh0000gn/T/ipykernel_94848/3790547094.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_simple.fillna({col: df_simple[col].mode()[0] for col in bool_cols}, inplace=True)


In [270]:
# Filling missing values with median for numerical features
numerical_features = df_simple.select_dtypes(include=['int64', 'float64']).columns
df_simple[numerical_features] = df_simple[numerical_features].fillna(df_simple[numerical_features].median())

In [271]:
# Create KMeans clusters
n_clusters = 300  # Adjust based on your data
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_simple['location_cluster'] = kmeans.fit_predict(df_simple[['latitude', 'longitude']])

In [277]:
df_simple.to_csv("datas/pre_onehot.csv", index=False)

In [272]:
# Encode categorical features
enc = OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=5, drop="if_binary", sparse_output=False)
categorical_features = [
    "neighbourhood_cleansed", 
    "neighbourhood_group_cleansed", 
    "host_response_time", "room_type", "location_cluster", "property_type"
]
enc.fit(df_simple[categorical_features])
encoded_array = enc.transform(df_simple[categorical_features])
# Retrieve the feature names after encoding
encoded_feature_names = enc.get_feature_names_out(categorical_features)

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_array, columns=encoded_feature_names, index=df_simple.index)

In [273]:
# Drop the original categorical columns
df_processed = df_simple.drop(columns=categorical_features)

# Concatenate the encoded features
df_processed = pd.concat([df_processed, encoded_df], axis=1)

In [217]:
# Select neighbourhood_cleansed and location_cluster columns
neighbourhood_features = [col for col in df_processed.columns if ('neighbourhood_cleansed' in col or 'neighbourhood_group_cleansed' in col)]
location_cluster_features = [col for col in df_processed.columns if 'location_cluster' in col]

# Combine these features into a single DataFrame
spatial_features = df_processed[neighbourhood_features + location_cluster_features]

In [161]:
from sklearn.decomposition import PCA

# Perform PCA to reduce dimensionality
pca = PCA(n_components=30)  # Adjust n_components based on explained variance
spatial_pca = pca.fit_transform(spatial_features)

# Create a new DataFrame with the PCA components
spatial_pca_df = pd.DataFrame(spatial_pca, columns=[f'spatial_pca_{i}' for i in range(1, 31)])

In [162]:
# Add PCA components to X_train
df_pca = df_processed.drop(columns=neighbourhood_features + location_cluster_features)  # Drop original features
df_pca = pd.concat([df_pca, spatial_pca_df], axis=1)

In [274]:
df_processed.drop(columns="id").iloc[0:15696,].to_csv("datas/df_1125v1_train.csv", index=False)

In [275]:
df_processed.iloc[15696:,].drop(columns="price").to_csv("datas/df_1125v1_test.csv", index=False)

In [163]:
df_processed.to_csv("datas/df_1122.csv", index=False)

In [139]:
df_pca.to_csv("datas/df_1122_pca.csv", index=False)

In [140]:
df_processed.head()

,price,latitude,longitude,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,accommodates,bathrooms,bedrooms,beds,availability_30,availability_60,availability_90,availability_365,instant_bookable,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,id,amenities_count,description_length,name_length,shared_bathrooms,has_washer,has_dryer,has_dishwasher,has_freezer,has_bbq_grill,has_hot_tub,has_pool,has_gym,has_balcony,has_backyard,neighbourhood_cleansed_Allerton,neighbourhood_cleansed_Arrochar,neighbourhood_cleansed_Arverne,neighbourhood_cleansed_Astoria,neighbourhood_cleansed_Bath Beach,neighbourhood_cleansed_Battery Park City,neighbourhood_cleansed_Bay Ridge,neighbourhood_cleansed_Baychester,neighbourhood_cleansed_Bayside,neighbourhood_cleansed_Bayswater,neighbourhood_cleansed_Bedford-Stuyvesant,neighbourhood_cleansed_Belle Harbor,neighbourhood_cleansed_Bellerose,neighbourhood_cleansed_Belmont,neighbourhood_cleansed_Bensonhurst,neighbourhood_cleansed_Bergen Beach,neighbourhood_cleansed_Boerum Hill,neighbourhood_cleansed_Borough Park,neighbourhood_cleansed_Briarwood,neighbourhood_cleansed_Brighton Beach,neighbourhood_cleansed_Bronxdale,neighbourhood_cleansed_Brooklyn Heights,neighbourhood_cleansed_Brownsville,neighbourhood_cleansed_Bull's Head,neighbourhood_cleansed_Bushwick,neighbourhood_cleansed_Cambria Heights,neighbourhood_cleansed_Canarsie,neighbourhood_cleansed_Carroll Gardens,neighbourhood_cleansed_Castle Hill,neighbourhood_cleansed_Castleton Corners,neighbourhood_cleansed_Chelsea,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_City Island,neighbourhood_cleansed_Civic Center,neighbourhood_cleansed_Claremont Village,neighbourhood_cleansed_Clason Point,neighbourhood_cleansed_Clifton,neighbourhood_cleansed_Clinton Hill,neighbourhood_cleansed_Co-op City,neighbourhood_cleansed_Cobble Hill,neighbourhood_cleansed_College Point,neighbourhood_cleansed_Columbia St,neighbourhood_cleansed_Concord,neighbourhood_cleansed_Concourse,neighbourhood_cleansed_Concourse Village,neighbourhood_cleansed_Coney Island,neighbourhood_cleansed_Corona,neighbourhood_cleansed_Crown Heights,neighbourhood_cleansed_Cypress Hills,neighbourhood_cleansed_DUMBO,neighbourhood_cleansed_Ditmars Steinway,neighbourhood_cleansed_Downtown Brooklyn,neighbourhood_cleansed_Dyker Heights,neighbourhood_cleansed_East Elmhurst,neighbourhood_cleansed_East Flatbush,neighbourhood_cleansed_East Harlem,neighbourhood_cleansed_East Morrisania,neighbourhood_cleansed_East New York,neighbourhood_cleansed_East Village,neighbourhood_cleansed_Eastchester,neighbourhood_cleansed_Edenwald,neighbourhood_cleansed_Edgemere,neighbourhood_cleansed_Elmhurst,neighbourhood_cleansed_Emerson Hill,neighbourhood_cleansed_Far Rockaway,neighbourhood_cleansed_Fieldston,neighbourhood_cleansed_Financial District,neighbourhood_cleansed_Flatbush,neighbourhood_cleansed_Flatiron District,neighbourhood_cleansed_Flatlands,neighbourhood_cleansed_Flushing,neighbourhood_cleansed_Fordham,neighbourhood_cleansed_Forest Hills,neighbourhood_cleansed_Fort Greene,neighbourhood_cleansed_Fort Hamilton,neighbourhood_cleansed_Fresh Meadows,neighbourhood_cleansed_Glendale,neighbourhood_cleansed_Gowanus,neighbourhood_cleansed_Gramercy,neighbourhood_cleansed_Graniteville,neighbourhood_cleansed_Grant City,neighbourhood_cleansed_Gravesend,neighbourhood_cleansed_Greenpoint,neighbourhood_cleansed_Greenwich Village,neighbourhood_cleansed_Harlem,neighbourhood_cleansed_Hell's Kitchen,neighbourhood_cleansed_Highbridge,ne